In [15]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('D:/Swansea/Uni/CSCM35 Data Mining/Coursework 2/data/GroceryStore.csv')

df.index.rename('TID', inplace=True)
df.rename(columns={'MILK,BREAD,BISCUIT' : 'Products'}, inplace=True)
#print(df)
trans_df = df.Products.str.split(',')


In [16]:
def prune(data,support):
    
    df = data[data.support_count >= support] 
    return df


In [17]:
def count_itemset(df, itemsets):   #Counting the group of elements in the dataset
                                    
    count_item = {}
    for item_set in itemsets:
        set_A = set(item_set)
        for row in trans_df:
            set_B = set(row)
        
            if set_B.intersection(set_A) == set_A: 
                if item_set in count_item.keys():
                    count_item[item_set] += 1
                
                else:
                    count_item[item_set] = 1
                
    data = pd.DataFrame()
    data['item_sets'] = count_item.keys()
    data['support_count'] = count_item.values()
    #print(data)
    
    return data


In [18]:
def count_item(transaction_items): #Individual element are being counted over here.
    
    count_ind_item = {}
    for row in transaction_items:
        for i in range(len(row)):
            if row[i] in count_ind_item.keys():
                count_ind_item[row[i]] += 1
            else:
                count_ind_item[row[i]] = 1
    
    data = pd.DataFrame()
    data['item_sets'] = count_ind_item.keys()
    data['support_count'] = count_ind_item.values()
    data = data.sort_values('item_sets')
    return data


In [19]:
def join(items_list):
    itemsets = []
    i = 1
    for entry in items_list:
        proceding_items = items_list[i:]
        for item in proceding_items:
            if(type(item) is str):
                if entry != item:
                    tuples = (entry, item)
                    itemsets.append(tuples)
            else:
                if entry[0:-1] == item[0:-1]:
                    tuples = entry+item[1:]
                    itemsets.append(tuples)
        i = i+1
        
    if(len(itemsets) == 0):
        return None
    return itemsets


In [20]:
def apriori(trans_data,support=3, con=0.5): #Implementing Apriori algorithm
    freq = pd.DataFrame()
    
    df = count_item(trans_data)
   
    while(len(df) != 0):
        
        df = prune(df, support)
    
        if len(df) > 1 or (len(df) == 1 and int(df.support_count >= support)):
            freq = df
            print(freq)
        itemsets = join(df.item_sets)
    
        if(itemsets is None):
            return freq
    
        df = count_itemset(trans_data, itemsets)
        #print(df)
    return df


In [8]:
freq_item_sets = apriori(trans_df, 2)

     item_sets  support_count
2      BISCUIT              7
5    BOURNVITA              4
1        BREAD             13
9         COCK              3
8       COFFEE              8
3   CORNFLAKES              6
6          JAM              2
7        MAGGI              5
0         MILK              5
10       SUGER              6
4          TEA              7
                item_sets  support_count
0        (BISCUIT, BREAD)              4
1         (BISCUIT, COCK)              2
2       (BISCUIT, COFFEE)              2
3   (BISCUIT, CORNFLAKES)              3
4        (BISCUIT, MAGGI)              2
5         (BISCUIT, MILK)              2
7          (BISCUIT, TEA)              2
8      (BOURNVITA, BREAD)              3
10     (BOURNVITA, SUGER)              2
11       (BOURNVITA, TEA)              2
13        (BREAD, COFFEE)              3
15           (BREAD, JAM)              2
16         (BREAD, MAGGI)              3
17          (BREAD, MILK)              4
18         (BREAD, SUGER)

In [9]:
freq_item_sets = apriori(trans_df, 3)

     item_sets  support_count
2      BISCUIT              7
5    BOURNVITA              4
1        BREAD             13
9         COCK              3
8       COFFEE              8
3   CORNFLAKES              6
7        MAGGI              5
0         MILK              5
10       SUGER              6
4          TEA              7
                item_sets  support_count
0        (BISCUIT, BREAD)              4
3   (BISCUIT, CORNFLAKES)              3
8      (BOURNVITA, BREAD)              3
13        (BREAD, COFFEE)              3
15         (BREAD, MAGGI)              3
16          (BREAD, MILK)              4
17         (BREAD, SUGER)              4
18           (BREAD, TEA)              4
19         (COCK, COFFEE)              3
21   (COFFEE, CORNFLAKES)              4
23        (COFFEE, SUGER)              4
30           (MAGGI, TEA)              4


In [21]:
freq_item_sets = apriori(trans_df, 4) 

     item_sets  support_count
2      BISCUIT              7
5    BOURNVITA              4
1        BREAD             13
8       COFFEE              8
3   CORNFLAKES              6
7        MAGGI              5
0         MILK              5
10       SUGER              6
4          TEA              7
               item_sets  support_count
0       (BISCUIT, BREAD)              4
14         (BREAD, MILK)              4
15        (BREAD, SUGER)              4
16          (BREAD, TEA)              4
17  (COFFEE, CORNFLAKES)              4
19       (COFFEE, SUGER)              4
26          (MAGGI, TEA)              4


In [22]:
def calculate_conf(value1, value2): #Counting confidence
    return round(int(value1)/int(value2) * 100, 2)


In [24]:
def strong_rules(freq_item_sets, threshold): #setting interesting rules 

    confidences = {}
    for row in freq_item_sets.item_sets:
        for i in range(len(row)):
            for j in range(len(row)):
                 if i != j:
                    tuples = (row[i], row[j])
                    conf = calculate_conf(freq_item_sets[freq_item_sets.item_sets == row].support_count, count_item(trans_df)[count_item(trans_df).item_sets == row[i]].support_count)
                    confidences[tuples] = conf

        
    conf_df = pd.DataFrame()
    conf_df['item_set'] = confidences.keys()
    conf_df['confidence'] = confidences.values()
    

    return conf_df[conf_df.confidence >= threshold]

print(strong_rules(freq_item_sets, 50.0))


                item_set  confidence
0       (BISCUIT, BREAD)       57.14
3          (MILK, BREAD)       80.00
5         (SUGER, BREAD)       66.67
7           (TEA, BREAD)       57.14
8   (COFFEE, CORNFLAKES)       50.00
9   (CORNFLAKES, COFFEE)       66.67
10       (COFFEE, SUGER)       50.00
11       (SUGER, COFFEE)       66.67
12          (MAGGI, TEA)       80.00
13          (TEA, MAGGI)       57.14


In [25]:
from functools import reduce
import operator

def interesting_rules(freq_item_sets):
    
    lifts = {}
    prob_of_items = []
  
    for row in freq_item_sets.item_sets:
        num_of_items = len(row)
        
        prob_all = freq_item_sets[freq_item_sets.item_sets == row].support_count / 18
        for i in range(num_of_items):
            prob_of_items.append(count_item(trans_df)[count_item(trans_df).item_sets == row[i]].support_count / 18)
        
        lifts[row] = round(float(prob_all / reduce(operator.mul, (np.array(prob_of_items)), 1)), 2)
        
        prob_of_items = []
        
    
    lifts_df = pd.DataFrame()
    lifts_df['Rules'] = lifts.keys()
    lifts_df['lift'] = lifts.values()
    return lifts_df

int_rules = interesting_rules(freq_item_sets)
print(int_rules)

                  Rules  lift
0      (BISCUIT, BREAD)  0.79
1         (BREAD, MILK)  1.11
2        (BREAD, SUGER)  0.92
3          (BREAD, TEA)  0.79
4  (COFFEE, CORNFLAKES)  1.50
5       (COFFEE, SUGER)  1.50
6          (MAGGI, TEA)  2.06


In [ ]:
'''
Code Reference:
KHAN, B. (2021, 11). Apriori Algorithm from Scratch. Retrieved from Kaggle:
    https://www.kaggle.com/code/rushikeshmmore/apriori-algorithm-on-online-retail-dataset
'''